In [ ]:
import kagglehub


path = kagglehub.dataset_download("olistbr/brazilian-ecommerce")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'brazilian-ecommerce' dataset.
Path to dataset files: /kaggle/input/brazilian-ecommerce


In [4]:
import os
import pandas as pd
import seaborn as sns
import numpy as np

In [5]:
print(os.listdir(path))

['olist_customers_dataset.csv', 'olist_sellers_dataset.csv', 'olist_order_reviews_dataset.csv', 'olist_order_items_dataset.csv', 'olist_products_dataset.csv', 'olist_geolocation_dataset.csv', 'product_category_name_translation.csv', 'olist_orders_dataset.csv', 'olist_order_payments_dataset.csv']


In [6]:
# Load datasets
customers = pd.read_csv(os.path.join(path, 'olist_customers_dataset.csv'))
items = pd.read_csv(os.path.join(path, 'olist_order_items_dataset.csv'))
payments = pd.read_csv(os.path.join(path, 'olist_order_payments_dataset.csv'))
reviews = pd.read_csv(os.path.join(path, 'olist_order_reviews_dataset.csv'))
orders = pd.read_csv(os.path.join(path, 'olist_orders_dataset.csv'))
products = pd.read_csv(os.path.join(path, 'olist_products_dataset.csv'))
sellers = pd.read_csv(os.path.join(path, 'olist_sellers_dataset.csv'))
category_translation = pd.read_csv(os.path.join(path, 'product_category_name_translation.csv'))
geolocation  = pd.read_csv(os.path.join(path, 'olist_geolocation_dataset.csv'))

In [7]:
reviews.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


In [ ]:

import matplotlib.pyplot as plt



df = orders.merge(reviews, on='order_id', how='inner')
print(f"Total orders with reviews: {len(df)}")
print(df[['order_id', 'order_status', 'review_score']].head())

date_columns = ['order_purchase_timestamp', 'order_approved_at',
                'order_delivered_carrier_date', 'order_delivered_customer_date',
                'order_estimated_delivery_date', 'review_creation_date',
                'review_answer_timestamp']

for col in date_columns:
    df[col] = pd.to_datetime(df[col], errors='coerce')

delivered_orders = df[df['order_status'] == 'delivered'].copy()

delivered_orders['actual_delivery_days'] = (
    delivered_orders['order_delivered_customer_date'] -
    delivered_orders['order_purchase_timestamp']
).dt.days

delivered_orders['estimated_delivery_days'] = (
    delivered_orders['order_estimated_delivery_date'] -
    delivered_orders['order_purchase_timestamp']
).dt.days

delivered_orders['delivery_delay_days'] = (
    delivered_orders['actual_delivery_days'] -
    delivered_orders['estimated_delivery_days']
)

print("\n=== Delivery Delay Statistics ===")
print(f"Average delay: {delivered_orders['delivery_delay_days'].mean():.2f} days")
print(f"Median delay: {delivered_orders['delivery_delay_days'].median():.2f} days")
print(f"Min delay: {delivered_orders['delivery_delay_days'].min():.2f} days")
print(f"Max delay: {delivered_orders['delivery_delay_days'].max():.2f} days")


Total orders with reviews: 99224
                           order_id order_status  review_score
0  e481f51cbdc54678b7cc49136f2d6af7    delivered             4
1  53cdb2fc8bc7dce0b6741e2150273451    delivered             4
2  47770eb9100c2d0c44946d9cf07ec65d    delivered             5
3  949d5b44dbf5de918fe9c16f97b45f8a    delivered             5
4  ad21c59c0840e6cb83a9ceb5573f8159    delivered             5

=== Delivery Delay Statistics ===
Average delay: -11.32 days
Median delay: -12.00 days
Min delay: -146.00 days
Max delay: 189.00 days


In [ ]:
delivered_orders['is_late'] = delivered_orders['delivery_delay_days'] > 0

delivered_orders['is_bad_review'] = delivered_orders['review_score'] <= 2

total_bad_reviews = delivered_orders['is_bad_review'].sum()

bad_reviews_late = delivered_orders[
    (delivered_orders['is_bad_review']) &
    (delivered_orders['is_late'])
].shape[0]

percentage_due_to_delay = (bad_reviews_late / total_bad_reviews) * 100

print("=== BAD REVIEW ANALYSIS ===")
print(f"Total bad reviews (1-2 stars): {total_bad_reviews}")
print(f"Bad reviews with late delivery: {bad_reviews_late}")
print(f"Percentage of bad reviews due to delay: {percentage_due_to_delay:.2f}%")

=== BAD REVIEW ANALYSIS ===
Total bad reviews (1-2 stars): 12347
Bad reviews with late delivery: 4096
Percentage of bad reviews due to delay: 33.17%


In [ ]:
late_orders = delivered_orders[delivered_orders['delivery_delay_days'] > 0].copy()
print(f"Orders that were actually late: {len(late_orders)} ({len(late_orders)/len(delivered_orders)*100:.1f}%)")
print(f"Average delay for late orders: {late_orders['delivery_delay_days'].mean():.2f} days")
print(f"Average review score for late orders: {late_orders['review_score'].mean():.2f}")

on_time_early = delivered_orders[delivered_orders['delivery_delay_days'] <= 0]
print(f"Average review score for on-time/early orders: {on_time_early['review_score'].mean():.2f}")

Orders that were actually late: 7183 (7.5%)
Average delay for late orders: 10.02 days
Average review score for late orders: 2.46
Average review score for on-time/early orders: 4.29


In [ ]:
delivered_orders['delivery_accuracy'] = pd.cut(
    delivered_orders['delivery_delay_days'].abs(),  
    bins=[0, 3, 7, 15, 30, float('inf')],
    labels=['Very Accurate (±3 days)',
            'Moderately Accurate (±4-7 days)',
            'Somewhat Inaccurate (±8-15 days)',
            'Very Inaccurate (±16-30 days)',
            'Extremely Inaccurate (>30 days)']
)

accuracy_analysis = delivered_orders.groupby('delivery_accuracy')['review_score'].agg(['mean', 'count'])
print(accuracy_analysis)

                                      mean  count
delivery_accuracy                                
Very Accurate (±3 days)           3.965828   7316
Moderately Accurate (±4-7 days)   4.019800  16212
Somewhat Inaccurate (±8-15 days)  4.209823  42879
Very Inaccurate (±16-30 days)     4.226217  26059
Extremely Inaccurate (>30 days)   3.950558   2508


/tmp/ipython-input-1390186999.py:13: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  accuracy_analysis = delivered_orders.groupby('delivery_accuracy')['review_score'].agg(['mean', 'count'])


In [ ]:

total_orders = len(delivered_orders)
late_orders = len(delivered_orders[delivered_orders['delivery_delay_days'] > 0])
avg_review_normal = delivered_orders[delivered_orders['delivery_delay_days'] <= 0]['review_score'].mean()
avg_review_late = delivered_orders[delivered_orders['delivery_delay_days'] > 0]['review_score'].mean()


preventable_late = late_orders * 0.5
improved_reviews = preventable_late * (avg_review_normal - avg_review_late)

print("=== BUSINESS IMPACT ANALYSIS ===")
print(f"Total orders: {total_orders}")
print(f"Late orders: {late_orders} ({late_orders/total_orders*100:.1f}%)")
print(f"Average review score (on-time): {avg_review_normal:.2f}")
print(f"Average review score (late): {avg_review_late:.2f}")
print(f"Review score drop when late: {avg_review_normal - avg_review_late:.2f} points")
print(f"\nIf we could prevent just 50% of late orders:")
print(f"→ {int(preventable_late)} orders would have better reviews")
print(f"→ Overall average review score would increase by: {improved_reviews/total_orders:.3f} points")

=== BUSINESS IMPACT ANALYSIS ===
Total orders: 96361
Late orders: 7183 (7.5%)
Average review score (on-time): 4.29
Average review score (late): 2.46
Review score drop when late: 1.83 points

If we could prevent just 50% of late orders:
→ 3591 orders would have better reviews
→ Overall average review score would increase by: 0.068 points
